# Analysis of Claremont Independent  
### Gigi Gedye & Eli Cohen


In [4]:
import pprint, os, csv, json, pickle, requests, html5lib
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup
from mozscape import Mozscape
pp = pprint.PrettyPrinter(indent=4)

#### Find All Articles
#### ! SKIP UNLESS UPDATING LIST !
This functoin takes will create a Python Dictionary of every Claremont Independent Article on their website.
The dictionary key is the article title, and the value is a list of the following:
* Date Published
* Author
* URL

In [ ]:
articles = {}
for page_num in range(1,68):
    base_url = 'http://claremontindependent.com/page/%d/' % page_num
    r = requests.get(base_url)
    soup = BeautifulSoup(r.text, "html5lib")

    for article in soup.find_all("article"):
        title = article.div.h3.a.string
        date = article.find('time').string
        author = article.find('em').string
        url = article.div.h3.a.get("href")
        
        entry = [date, author, url]
        articles[title] = entry
        print(entry)
    
with open('articles.json', 'w') as f:
    json.dump(articles, f)

#### Create Text Corpus
#### ! SKIP UNLESS UPDATING LIST !
Take list of articles, and retrieve article text from every page

In [12]:
with open('articles.json', 'r') as f:
    articles = json.load(f)
    
corpus = []
for article in articles: 
    r = requests.get(articles.get(article)[2])
    soup = BeautifulSoup(r.text, "html5lib")
    body = soup.find(attrs={"class": "entry-content"})
    paragraphs = body.get_text()
    text, seperator, attribution = paragraphs.partition("\n\n")
    
    corpus.append(text)

with open('corpus.json','w') as f:
    json.dump(corpus, f)

#### Get Article Backlinks
Using the Mozscape API, this function will create a python list of every webpage which "backlinks" to the webpage which we feed in.

In [5]:
with open('articles.json', 'r') as f:
    articles_dict = json.load(f)

client = Mozscape('mozscape-14bcb34ac8', '73bfb82e39e7868337668d339b61b805')
for key, value in articles_dict.items(): 
    links_info = client.links( value[2],
                               scope = 'page_to_page',
                               sort = 'domains_linking_page',
                               filters = ['external'],
                               limit = 25,
                               offset = 25,
                               sourceCols = 5)
    
    if len(links_info) != 0:
        pp.pprint(links_info)
    
with open('articles_dict.json', 'w') as f:
    json.dump(articles_dict, f)

TypeError: links() got an unexpected keyword argument 'limit'